<a href="https://colab.research.google.com/github/ycaparicioa/MetNumUN2023I/blob/main/Lab7/Week4iterativeMethodsForLinearSystemsGroup2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
import numpy as np

def jacobi_iteration(A, b, n_iter):
    n = A.shape[0]
    
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    
    D = np.diag(diag_1d)
    invD = np.diag(1. / diag_1d)
    BB = invD @ B
    c = invD @ b
    
    x0 = np.ones(n)
    x = x0
    
    for _ in range(n_iter):
        x = BB @ x + c
    
    return x

# Test the Jacobi iteration
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15] * n)
b = rndm.uniform(size=n)
n_iter = 50

x = jacobi_iteration(A, b, n_iter)
print("Solution x:", x)

# Manipulate diagonal elements of matrix A and check convergence
diag_values = [1, 0.1, 0.01, 0.001]
norm_values = []

for value in diag_values:
    modified_A = A.copy()
    np.fill_diagonal(modified_A, value)
    x = jacobi_iteration(modified_A, b, n_iter)
    norm_B = np.linalg.norm(-modified_A + np.diag(np.diag(modified_A)))
    norm_values.append(norm_B)

print("Diagonal values of A:", diag_values)
print("Norm of B:", norm_values)


Solution x: [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Diagonal values of A: [1, 0.1, 0.01, 0.001]
Norm of B: [5.629112458567935, 5.629112458567935, 5.629112458567935, 5.629112458567935]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
import numpy as np

def seidel_iteration(A, b, n_iter):
    n = A.shape[0]
    x = np.ones(n)
    
    for _ in range(n_iter):
        for i in range(n):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]
    
    return x

# Test the Seidel's iteration
rndm = np.random.RandomState(1234)
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15] * n)
b = rndm.uniform(size=n)
n_iter = 50

x = seidel_iteration(A, b, n_iter)
print("Solution x:", x)

# Calculate the norm of the iteration matrix for different iterations
norm_values = []
iter_values = list(range(1, 51))

for n_iter in iter_values:
    x = seidel_iteration(A, b, n_iter)
    norm_B = np.linalg.norm(-A + np.tril(A))
    norm_values.append(norm_B)

print("Number of iterations:", iter_values)
print("Norm of the iteration matrix:", norm_values)


Solution x: [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Number of iterations: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Norm of the iteration matrix: [4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.276315792082548, 4.2763157920

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
def minimum_residual_scheme(A, b, x0, tol=1e-6, max_iter=100):
  """
  Solves the linear system Ax = b using the minimum residual scheme.

  Args:
    A: The coefficient matrix.
    b: The right-hand side vector.
    x0: The initial guess.
    tol: The tolerance for convergence.
    max_iter: The maximum number of iterations.

  Returns:
    The solution x.
  """

  x = x0
  r = b - A @ x
  for i in range(max_iter):
    τ = 1
    while True:
      x_new = x + τ * r
      r_new = b - A @ x_new
      if np.linalg.norm(r_new) < np.linalg.norm(r) * (1 - τ):
        break
      τ *= 0.5
    x = x_new
    r = r_new

    if np.linalg.norm(r) < tol:
      break

  return x
